In [1]:
from pprint import pprint

import numpy as np
import pandas as pd

from tqdm.auto import tqdm

from rectools import Columns
from rectools.dataset import Interactions, Dataset
from rectools.metrics import MRR, MAP, Precision, Recall, MeanInvUserFreq, Serendipity, calc_metrics
from rectools.models import RandomModel, PopularModel
from rectools.model_selection import TimeRangeSplitter


# Загрузка и чтение данных

In [2]:
%%time
# !wget https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip -O ../data/data_original.zip
# !uzip ../data/data_original.zip

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 9.3 µs


In [3]:
interactions_df = pd.read_csv("../data/data_original/interactions.csv")
print(interactions_df.shape)   
interactions_df.head()

(5476251, 5)


,user_id,item_id,last_watch_dt,total_dur,watched_pct
0,176549,9506,2021-05-11,4250,72.0
1,699317,1659,2021-05-29,8317,100.0
2,656683,7107,2021-05-09,10,0.0
3,864613,7638,2021-07-05,14483,100.0
4,964868,9506,2021-04-30,6725,100.0


In [4]:
items_df = pd.read_csv("../data/data_original/items.csv")
print(items_df.shape)   
items_df.head()

(15963, 14)


,item_id,content_type,title,title_orig,release_year,genres,countries,for_kids,age_rating,studios,directors,actors,description,keywords
0,10711,film,Поговори с ней,Hable con ella,2002.0,"драмы, зарубежные, детективы, мелодрамы",Испания,NaN,16.0,NaN,Педро Альмодовар,"Адольфо Фернандес, Ана Фернандес, Дарио Гранди...",Мелодрама легендарного Педро Альмодовара «Пого...,"Поговори, ней, 2002, Испания, друзья, любовь, ..."
1,2508,film,Голые перцы,Search Party,2014.0,"зарубежные, приключения, комедии",США,NaN,16.0,NaN,Скот Армстронг,"Адам Палли, Брайан Хаски, Дж.Б. Смув, Джейсон ...",Уморительная современная комедия на популярную...,"Голые, перцы, 2014, США, друзья, свадьбы, прео..."
2,10716,film,Тактическая сила,Tactical Force,2011.0,"криминал, зарубежные, триллеры, боевики, комедии",Канада,NaN,16.0,NaN,Адам П. Калтраро,"Адриан Холмс, Даррен Шалави, Джерри Вассерман,...",Профессиональный рестлер Стив Остин («Все или ...,"Тактическая, сила, 2011, Канада, бандиты, ганг..."
3,7868,film,45 лет,45 Years,2015.0,"драмы, зарубежные, мелодрамы",Великобритания,NaN,16.0,NaN,Эндрю Хэй,"Александра Риддлстон-Барретт, Джеральдин Джейм...","Шарлотта Рэмплинг, Том Кортни, Джеральдин Джей...","45, лет, 2015, Великобритания, брак, жизнь, лю..."
4,16268,film,Все решает мгновение,NaN,1978.0,"драмы, спорт, советские, мелодрамы",СССР,NaN,12.0,Ленфильм,Виктор Садовский,"Александр Абдулов, Александр Демьяненко, Алекс...",Расчетливая чаровница из советского кинохита «...,"Все, решает, мгновение, 1978, СССР, сильные, ж..."


In [5]:
interactions_df["user_id"].nunique(), interactions_df["item_id"].nunique()

(962179, 15706)

In [6]:
interactions_df["datetime"] = interactions_df["last_watch_dt"]
interactions_df["weight"] = 1
interactions_df.head()

,user_id,item_id,last_watch_dt,total_dur,watched_pct,datetime,weight
0,176549,9506,2021-05-11,4250,72.0,2021-05-11,1
1,699317,1659,2021-05-29,8317,100.0,2021-05-29,1
2,656683,7107,2021-05-09,10,0.0,2021-05-09,1
3,864613,7638,2021-07-05,14483,100.0,2021-07-05,1
4,964868,9506,2021-04-30,6725,100.0,2021-04-30,1


In [7]:
interactions = Interactions(interactions_df)

# Обучение и валидация
### Расчёт метрик (5 баллов)
1. Функция будет принимать на вход:  
- 1.1 Словарь с инициализированными моделями  +
- 1.2 Словарь с инициализированными метриками  +
- 1.3 Инициализированный Splitter для кросс-валидации +  
- 1.4 Количество рекомендаций для генерации (K)  +
2. Реализация обучения и валидации:  
- 1.1 Создаем RecTools Dataset через метод construct на train взаимодействиях  для каждого фолда  +
- 1.2 Обучаем модель (не забываем сделать deepcopy), рекоменуем K айтемов для каждого юзера, считаем метрики на test  +
- 1.3 Дополнительно логируем время обучения  +
- 1.4 Сохраняем метрики в отчёт  +
3. Результат оборачиваем в pandas DataFrame и усредняем по фолдам  +

In [8]:
def train_and_validate_model(interactions, models, metrics, splitter, K_RECOS, n_splits):

    results = []

    fold_iterator = splitter.split(interactions, collect_fold_stats=True)

    for train_ids, test_ids, fold_info in tqdm((fold_iterator), total=n_splits): # Kernel error when use
    # for train_ids, test_ids, fold_info in fold_iterator:
        print(f"\n==================== Fold {fold_info['i_split']}")
        pprint(fold_info)

        df_train = interactions.df.iloc[train_ids]
        dataset = Dataset.construct(df_train)

        df_test = interactions.df.iloc[test_ids][Columns.UserItem]
        test_users = np.unique(df_test[Columns.User])
        
        catalog = df_train[Columns.Item].unique()

        for model_name, model in models.items():
            model.fit(dataset)
            recos = model.recommend(
                users=test_users,
                dataset=dataset,
                k=K_RECOS,
                filter_viewed=True,
            )
            metric_values = calc_metrics(
                metrics,
                reco=recos,
                interactions=df_test,
                prev_interactions=df_train,
                catalog=catalog,
            )
            res = {"fold": fold_info["i_split"], "model": model_name}
            res.update(metric_values)
            results.append(res)
    return results

In [9]:
%%time

n_splits = 3

splitter = TimeRangeSplitter(
    test_size="14D",
    n_splits=n_splits,
    filter_already_seen=True,
    filter_cold_items=True,
    filter_cold_users=True,
)

models = {
    "random": RandomModel(random_state=32),
    "popular": PopularModel(),
}

metrics = {
    "prec@1": Precision(k=1),
    "prec@5": Precision(k=5),
    "prec@10": Precision(k=10),
    "recall@1": Recall(k=1),
    "recall@5": Recall(k=5),
    "recall@10": Recall(k=10),
    "novelty@1": MeanInvUserFreq(k=1),
    "novelty@5": MeanInvUserFreq(k=5),
    "novelty@10": MeanInvUserFreq(k=10),
    "serendipity@1": Serendipity(k=1),
    "serendipity@5": Serendipity(k=5),
    "serendipity@10": Serendipity(k=10),
    "MAP@1": MAP(k=1),
    "MAP@5": MAP(k=5),
    "MAP@10": MAP(k=10),
    "MRR@1": MRR(k=1),
    "MRR@5": MRR(k=5),
    "MRR@10": MRR(k=10),
}

K_RECOS = 10


results = train_and_validate_model(interactions, models, metrics, splitter, K_RECOS, n_splits)

  0%|          | 0/3 [00:00<?, ?it/s]


==================== Fold 0
{'end': Timestamp('2021-07-26 00:00:00', freq='14D'),
 'i_split': 0,
 'start': Timestamp('2021-07-12 00:00:00', freq='14D'),
 'test': 398993,
 'test_items': 7394,
 'test_users': 122488,
 'train': 3239125,
 'train_items': 14730,
 'train_users': 646423}

==================== Fold 1
{'end': Timestamp('2021-08-09 00:00:00', freq='14D'),
 'i_split': 1,
 'start': Timestamp('2021-07-26 00:00:00', freq='14D'),
 'test': 458757,
 'test_items': 7711,
 'test_users': 135624,
 'train': 3892558,
 'train_items': 15085,
 'train_users': 742256}

==================== Fold 2
{'end': Timestamp('2021-08-23 00:00:00', freq='14D'),
 'i_split': 2,
 'start': Timestamp('2021-08-09 00:00:00', freq='14D'),
 'test': 521381,
 'test_items': 7705,
 'test_users': 151629,
 'train': 4649162,
 'train_items': 15415,
 'train_users': 850489}
CPU times: user 2min 53s, sys: 3.72 s, total: 2min 56s
Wall time: 2min 57s


In [10]:
pivot_results = pd.DataFrame(results).drop(columns="fold").groupby(["model"], sort=False).agg(["mean", "std"])
mean_metric_subset = [(metric, agg) for metric, agg in pivot_results.columns if agg == 'mean']
(
    pivot_results.style
    .highlight_min(subset=mean_metric_subset, color='red', axis=0)
    .highlight_max(subset=mean_metric_subset, color='green', axis=0)
)

# Визуализация рекомендаций и действительности

### Визуальный анализ (5 баллов)
1. Функция будет принимать на вход:  
- 1.1 Инициализированную (и обученную!) модель  +
- 1.2 Датасет  +
- 1.3 Список отобранных user_id для просмотра  +
- 1.4 item_data - данные об айтемах, которые важно отразить для визуального анализа (напр. название)  +
2. Реализация:  
- 1.1 Генерим рекомендации для отобранных user_id  
- 1.2 Получаем из датасета истории просмотров юзеров  
- 1.3 Отображаем в любом удобном виде, который позволит смотреть на историю просмотра юзера + на его рекомендации от модели  

In [11]:
model = RandomModel(random_state=32)
dataset = Dataset.construct(interactions.df)
model.fit(dataset)

In [12]:
def visualize_pred_and_gt(model, interactions_dataset, user_ids, item_data, item_data_col, K_RECOS):

    dataset = Dataset.construct(interactions_dataset)
    if "item_id" not in item_data_col:
        item_data_col.insert(0,"item_id")
        
    recos = model.recommend(
        users=user_ids,
        dataset=dataset,
        k=K_RECOS,
        filter_viewed=True,
        add_rank_col=True,
    )
    
    for user in user_ids:
        print("История пользователя:")
        display(interactions_dataset[interactions_dataset.user_id == user].merge(item_data[item_data_col], on='item_id').merge(interactions_dataset.groupby('item_id')['user_id'].count().reset_index().rename(columns={'user_id': 'interaction_count'}), on='item_id'))
        print("Рекомендации:")
        display(recos[recos.user_id == user].merge(item_data[item_data_col], on='item_id').merge(interactions_dataset.groupby('item_id')['user_id'].count().reset_index().rename(columns={'user_id': 'interaction_count'}), on='item_id'))


In [13]:
user_ids = [666262, 672861, 955527]
K_RECOS = 10
items_data_col = ["content_type", "title", "genres"]

visualize_pred_and_gt(model, interactions.df, user_ids, items_df, items_data_col, K_RECOS)

История пользователя:


,user_id,item_id,last_watch_dt,total_dur,watched_pct,datetime,weight,content_type,title,genres,interaction_count
0,666262,7957,2021-05-12,2052,32.0,2021-05-12,1.0,film,Последний викинг,"боевики, историческое, приключения",746
1,666262,4785,2021-05-12,1946,28.0,2021-05-12,1.0,film,Робин Гуд: Начало,"боевики, триллеры, приключения",485
2,666262,12981,2021-05-14,10292,100.0,2021-05-14,1.0,film,Томирис,"боевики, драмы, историческое, военные",10370


Рекомендации:


,user_id,item_id,score,rank,content_type,title,genres,interaction_count
0,666262,10101,10,1,series,Возвращение Будулая,мелодрамы,99
1,666262,619,9,2,film,Новые приключения Аладдина (жестовым языком),"зарубежные, комедии",1
2,666262,12618,8,3,film,Пропавшая грамота,"фэнтези, комедии",51
3,666262,5967,7,4,series,Братья вне игры,"драмы, спорт",262
4,666262,4041,6,5,film,Фрилансеры,"криминал, детективы, драмы, зарубежные, боевики",19
5,666262,5701,5,6,film,Алые паруса: Новая история,"комедии, мелодрамы",4
6,666262,9738,4,7,series,Женщина в беде 3,"детективы, мелодрамы",2
7,666262,15247,3,8,film,Гордость и предубеждение,"драмы, мелодрамы",150
8,666262,10004,2,9,film,Болванчики,"мультфильм, приключения, комедии",51
9,666262,2816,1,10,film,Избави нас от лукавого,"ужасы, триллеры, детективы",1370


История пользователя:


...

,user_id,item_id,score,rank,content_type,title,genres,interaction_count
0,672861,9457,10,1,film,Комната (жестовым языком),"драмы, зарубежные, триллеры",5
1,672861,15730,9,2,series,Твое подтянутое тело,фитнес,2
2,672861,473,8,3,series,Кто такой Букабу?,"развлекательные, для детей, документальное",15
3,672861,12736,7,4,film,Палач,"драмы, зарубежные, комедии",3
4,672861,3927,6,5,film,Помни меня,"драмы, мелодрамы",2982
5,672861,3300,5,6,film,Антилопа Гну. Южная Африка,документальное,8
6,672861,5334,4,7,series,Boys and Toys,no_genre,3
7,672861,14273,3,8,film,Влюбленный скорпион,"драмы, зарубежные, спорт, триллеры, мелодрамы",2
8,672861,3087,2,9,series,Жуки - караоке,no_genre,1
9,672861,4416,1,10,film,Питер,"фэнтези, приключения",33


История пользователя:


,user_id,item_id,last_watch_dt,total_dur,watched_pct,datetime,weight,content_type,title,genres,interaction_count
0,955527,1183,2021-06-02,40,1.0,2021-06-02,1.0,film,Стань легендой! Бигфут Младший,"мультфильм, фэнтези, приключения, комедии",1587
1,955527,13371,2021-05-04,686,11.0,2021-05-04,1.0,film,Пеле: Рождение легенды,"драмы, спорт, биография",945
2,955527,4725,2021-06-02,255,4.0,2021-06-02,1.0,film,Лобановский навсегда,"спорт, биография, документальное",683
3,955527,1238,2021-06-02,556,7.0,2021-06-02,1.0,film,Диего Марадона,"спорт, биография, документальное",691


Рекомендации:


,user_id,item_id,score,rank,content_type,title,genres,interaction_count
0,955527,496,10,1,series,Воскресший Эртугрул,"боевики, драмы, приключения",6167
1,955527,4205,9,2,series,Дело гастронома №1 (Операция Беркут),"драмы, русские",1
2,955527,10822,8,3,film,Она защищает Родину,"драмы, советские, военные",2
3,955527,10914,7,4,film,Великолепная,"зарубежные, комедии, мелодрамы",3
4,955527,3999,6,5,film,Джиперс криперс,"ужасы, триллеры",648
5,955527,15756,5,6,film,Ремнант: Всё ещё вижу тебя (жестовым языком),"фантастика, зарубежные, триллеры",2
6,955527,14961,4,7,film,Битва за Землю,"боевики, ужасы, фантастика, триллеры",2032
7,955527,13734,3,8,film,Сексуальный массаж и Фантазии,для взрослых,31
8,955527,3407,2,9,film,Черный капитан,"боевики, русские, военные",1
9,955527,14614,1,10,film,Настя,"мелодрамы, комедии",2
